In [29]:
from secret import SECRET
import networkx as nx
import re
import json
import random
from urllib.parse import quote
import requests
import folium as f
import pprint
from demo import DATASET

BASE_URL = "https://maps.googleapis.com/maps/api/directions/json?"
URL_END = "&mode=bicycling&key=" + SECRET
WAYPOINT_MAX = 23 # In case we ever use waypoints
 

In [30]:
import pickle
with open('./data/07' + DATASET + '_yes_central_square_final_edges.pkl', 'rb') as file:
    edge_data = pickle.load(file)

with open('./data/07_hardcoded_point_names.pkl', 'rb') as file:
    hardcoded_point_names = pickle.load(file)
    
with open('./data/08_' + DATASET + '_previous_urls.pkl', 'rb') as file:
    previous_urls = pickle.load(file)

In [3]:
[x for x in edge_data if x[0] == [42.369192, -71.0730914] or x[1] == [42.369192, -71.0730914]]

[]

In [4]:
edge_data

[[[42.3227638, -71.0999462], [42.3252094, -71.0749335]],
 [[42.310066, -71.107549], [42.3227638, -71.0999462]],
 [[42.3737246, -71.1008111], [42.3742679, -71.1188142]],
 [[42.3742679, -71.1188142], [42.379635, -71.096387]],
 [[42.379635, -71.096387], [42.3737246, -71.1008111]],
 [[42.342193, -71.1212516], [42.3632611, -71.1296873]],
 [[42.3524881, -71.0553466], [42.3597038, -71.0520451]],
 [[42.3297015, -71.0568123], [42.3524881, -71.0553466]],
 [[42.3525933, -71.0673168], [42.3524881, -71.0553466]],
 [[42.3252094, -71.0749335], [42.3326217, -71.0720415]],
 [[42.3326217, -71.0720415], [42.3297015, -71.0568123]],
 [[42.3326217, -71.0720415], [42.3525933, -71.0673168]],
 [[42.3326217, -71.0720415], [42.3464752, -71.0909098]],
 [[42.3464752, -71.0909098], [42.3525933, -71.0673168]],
 [[42.3570628, -71.092537], [42.3521151, -71.0897845]],
 [[42.3521151, -71.0897845], [42.3464752, -71.0909098]],
 [[42.3521151, -71.0897845], [42.3525933, -71.0673168]],
 [[42.3157205, -71.1174339], [42.322763

In [5]:
def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs
       WRITTEN BY mgd722
    '''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index += 1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))

    return coordinates

In [6]:
def generate_url(origin, dest):
    origin_param = "&" + "origin=" +quote((str(origin['lat']) + ',' + str(origin['lon'])))
    dest_param = "&" + "destination=" + quote((str(dest['lat']) + ',' + str(dest['lon'])))

    url = BASE_URL + origin_param + dest_param
    return url



In [7]:
for edge in edge_data:
    origin = {'lat': edge[0][0], 'lon': edge[0][1]}
    dest = {'lat': edge[1][0], 'lon': edge[1][1]}
    
    right = generate_url(origin, dest)
    left = generate_url(dest, origin)
    
    if previous_urls.get(right) is None:
        print('hit')
        previous_urls[right] = {} 
    if previous_urls.get(left) is None:
        previous_urls[left] = {}
    

In [8]:
for url, v in previous_urls.items():
    if v.get('response', None) is None:
        print('hit')
        route_data = requests.get(url + URL_END).json()
        previous_urls[url]['response'] = route_data
with open('./data/08_' + DATASET + '_previous_urls.pkl', 'wb') as file:
    pickle.dump(previous_urls, file)

In [9]:
unique_nodes = set()

for edge in edge_data:
    origin = (edge[0][0], edge[0][1])
    dest = (edge[1][0], edge[1][1])
    unique_nodes.add(origin)
    unique_nodes.add(dest)


In [10]:
unique_nodes

{(42.310066, -71.107549),
 (42.3157205, -71.1174339),
 (42.3227638, -71.0999462),
 (42.3252094, -71.0749335),
 (42.3297015, -71.0568123),
 (42.3317919, -71.1134095),
 (42.3326217, -71.0720415),
 (42.3353723, -71.090063),
 (42.342193, -71.1212516),
 (42.3448431, -71.1042418),
 (42.3464752, -71.0909098),
 (42.3504399, -71.1108926),
 (42.3521151, -71.0897845),
 (42.3524881, -71.0553466),
 (42.3525933, -71.0673168),
 (42.3539609, -71.1109653),
 (42.3570628, -71.092537),
 (42.3597038, -71.0520451),
 (42.3610239, -71.0715613),
 (42.3622709, -71.1135298),
 (42.3624477, -71.0843118),
 (42.3632611, -71.1296873),
 (42.3655877, -71.1040004),
 (42.369837, -71.112607),
 (42.3737246, -71.1008111),
 (42.3742679, -71.1188142),
 (42.379635, -71.096387)}

In [27]:
def draw_map():
    m = f.Map(location = [42.3624477, -71.0843118], zoom_start=14)
#     f.TileLayer('cartodbpositron').add_to(m)
#     f.TileLayer('openstreetmap').add_to(m)
    

    for node in unique_nodes:
        lat = node[0]
        lon = node[1]
        
        size = 25
        color = "green"
        
        node_number = hardcoded_point_names[(lat, lon)]
        
        f.Marker(location=[lat,lon],
          # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
          icon=f.DivIcon(html=f"""<div style="border:1px solid {color}; text-align: center; vertical-align: middle; line-height: {size}px; background-color: white; width: {size}px; height: {size}px; border-radius: {size}px; font-family: sans-serif; color: {color};">{node_number}</div>"""),
          popup = str(lat) + ", " + str(lon),
         ).add_to(m)

    
    for edge in edge_data:
        origin = {'lat': edge[0][0], 'lon': edge[0][1]}
        dest = {'lat': edge[1][0], 'lon': edge[1][1]}
        
        route_to_dest = generate_url(origin, dest)
        route_to_origin = generate_url(dest, origin)
        
        poly_to_dest = previous_urls[route_to_dest]
        poly_to_origin = previous_urls[route_to_origin]
    
        steps_to_dest = poly_to_dest['response']['routes'][0]['legs'][0]['steps']
        steps_to_origin = poly_to_origin['response']['routes'][0]['legs'][0]['steps']
        
        for step in steps_to_dest + steps_to_origin:
            encoded_polyline = step["polyline"]["points"]
            polyline = decode_polyline(encoded_polyline)

#             if (origin['lat'] == 42.3655877 or 
#                 dest['lat'] == 42.3655877) and (origin['lon'] == -71.1040004 or 
#                                                 dest['lon'] == -71.1040004):
                
        
#                 m.add_child(f.PolyLine(locations=polyline, opacity=1, color='magenta'))
#             else:
            m.add_child(f.PolyLine(locations=polyline, weight=6, opacity=1, color=color))
    
    
 
    bike_icon = f.features.CustomIcon("./img/bike2.png",icon_size=(98, 62))
    f.Marker(draggable=True, location=(42.3624477, -71.0843118),icon=bike_icon).add_to(m)

    
    return m

In [28]:
f.TileLayer(opacity=0.2).add_to(m)

m = draw_map()
# f.opacity=.2
# f.TileLayer(opacity=0.5).add_to(m)
# f.TileLayer('stamenterrain').add_to(m)
f.TileLayer('stamenwatercolor').add_to(m)
m